论文阅读 + 复现：（LRCNs）Long-term Recurrent Convolutional Networks for Visual Recognition and Description:


https://blog.csdn.net/qq_36627158/article/details/114026519

GitHub:

https://github.com/BizhuWu/LRCN_PyTorch

In [1]:
# clone git repo
!git clone https://github.com/BizhuWu/LRCN_PyTorch
!mv LRCN_PyTorch/* .
!rm -rf LRCN_PyTorch/

Cloning into 'LRCN_PyTorch'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 36 (delta 17), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [2]:
# Download UCF101 dataset and split list
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
!wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip

--2021-06-08 05:29:19--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  11.5MB/s    in 9m 43s  

2021-06-08 05:39:03 (11.3 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]

--2021-06-08 05:39:03--  https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 113943 (111K) [application/zip]
Saving to: ‘UCF101TrainTestSplits-RecognitionTask.zip’

UCF

In [4]:
# Extract and organize video data and split list
!mkdir -p data/UCF101/UCF-101/
!unrar e UCF101.rar data/UCF101/UCF-101/
import os
for files in os.listdir('data/UCF101/UCF-101/'):
    foldername = files.split('_')[1]
    os.system("mkdir -p data/UCF101/UCF-101/" + foldername)
    os.system("mv data/UCF101/UCF-101/"+ files + " data/UCF101/UCF-101/" + foldername)
# !rm data/UCF101/UCF-101/*.avi

!mkdir -p data/UCF101TrainTestSplits-RecognitionTask
!unzip UCF101TrainTestSplits-RecognitionTask.zip -d data/UCF101TrainTestSplits-RecognitionTask

Archive:  UCF101TrainTestSplits-RecognitionTask.zip
   creating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/classInd.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist01.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist02.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist03.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist01.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist02.txt  
  inflating: data/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/trainlist03.txt  


In [1]:
!nvidia-smi
!which python
!which apt-get
!which pip

Tue Jun  8 11:44:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.80       Driver Version: 460.80       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   47C    P2    54W / 280W |    247MiB / 11178MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Install dependencies for video module in pytorch to work
# ! apt-get install libavformat-dev libavdevice-dev
! pip install av==6.2.0

  Using cached av-6.2.0.tar.gz (2.0 MB)
  Created wheel for av: filename=av-6.2.0-cp37-cp37m-linux_x86_64.whl size=5120008 sha256=26de819f8937ceaccce65b2d65fb18c817000d98b26172e2708c415314d67634
  Stored in directory: /home/chihyuan/.cache/pip/wheels/bd/83/e3/93ca73311d72bebad4a806dc9a337f7d9e17dc5ec3fd4b58a8
Successfully built av


In [7]:
from LoadUCF101DataByTorch import trainset_loader, testset_loader
from model import LRCN
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt



EPOCH = 10
LEARNING_RATE = 0.003
MOMENTUM = 0.9
GAMMA = 0.5
STEP_SIZE = 1

print("HELLO")

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)



model = LRCN().to(device)



optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LEARNING_RATE,
    momentum=MOMENTUM
)
# scheduler = torch.optim.lr_scheduler.StepLR(
#     optimizer,
#     step_size=STEP_SIZE,
#     gamma=GAMMA
# )



def save_checkpoint(path, model, optimizer):
    state = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, path)



def train(epoch):
    iteration = 0
    loss_plt=[]


    for i in range(epoch):
        model.train()
        # print('current lr', scheduler.get_last_lr())
        
        for index, data in enumerate(trainset_loader):
            video_clips, label = data

            video_clips = video_clips.to(device)
            label = label.to(device)

            optimizer.zero_grad()

            output = model(video_clips)

            loss = F.cross_entropy(output, label)

            loss_plt.append(loss.item())

            loss.backward()
            optimizer.step()

            iteration += 1

            print("Epoch:", i, "/", epoch-1, "\tIteration:", index, "/", len(trainset_loader)-1, "\tLoss: " + str(loss.item()))
            with open('log.txt', 'a') as f:
                f.write("Epoch: " + str(i) + "/" + str(epoch-1) + "\tIteration:" + str(index) + "/" + str(len(trainset_loader)-1) + "\tLoss: " + str(loss.item()) + "\n")

        save_checkpoint('model/checkpoint-%i.pth' % iteration, model, optimizer)

        test(i)

        # scheduler.step()

    save_checkpoint('model/checkpoint-%i.pth' % iteration, model, optimizer)

    plt.figure()
    plt.plot(loss_plt)
    plt.title('Loss')
    plt.xlabel('Iteration')
    plt.ylabel('')
    plt.show()



def test(i_epoch):
    model.eval()

    correct = 0

    with torch.no_grad():
        for index, data in enumerate(testset_loader):
            video_clips, label = data

            video_clips = video_clips.to(device)
            label = label.to(device)

            output = model(video_clips)

            max_value, max_index = output.max(1, keepdim=True)
            correct += max_index.eq(label.view_as(max_index)).sum().item()

    print("Accuracy: " + str(correct * 1.0 * 100 / len(testset_loader.dataset)))
    with open('log.txt', 'a') as f:
        f.write("Epoch " + str(i_epoch) + "'s Accuracy: " + str(correct * 1.0 * 100 / len(testset_loader.dataset)) + "\n")



HELLO
cuda


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/chihyuan/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

/home/chihyuan/anaconda3/envs/ntupac21s/lib/python3.7/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
train(EPOCH)

In [13]:
iteration = 0
save_checkpoint('model/checkpoint-%i.pth' % iteration, model, optimizer)